In [ ]:
!pip install rdkit kaggle

In [ ]:
!pip install pyvis

In [3]:
import time
import sys
import numpy      as np
import pandas     as pd
import matplotlib.pyplot as plt
import seaborn    as sns
import networkx   as nx  # For creating and manipulating graph structures

from   zipfile    import ZipFile
from   datetime   import datetime
from collections import Counter
import random
import re
import psutil
import copy
import json
import os

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdmolops
from tqdm       import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset
from torch.optim import lr_scheduler
from torch.distributions import Normal

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics         import accuracy_score, mean_absolute_error, r2_score
from sklearn.preprocessing   import StandardScaler, MinMaxScaler
from sklearn.decomposition   import TruncatedSVD
from sklearn.model_selection import KFold

In [4]:
api_token = {"username":"alji1305","key":"4703b0a6c6a8543e51a6de4131ab05cb"}

with open('kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

with open('kaggle.json', 'r') as file:
    kaggle_credentials = json.load(file)

username = kaggle_credentials['username']
api_key  = kaggle_credentials['key']


os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY']      = api_key

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d basu369victor/zinc250k

 81% 7.00M/8.60M [00:00<00:00, 50.3MB/s]
100% 8.60M/8.60M [00:00<00:00, 39.9MB/s]


In [6]:
with ZipFile('zinc250k.zip') as zp:
    zinc_data = pd.read_csv(zp.open('250k_rndm_zinc_drugs_clean_3.csv'))

adjusted_smiles  = []
for index, smile in enumerate(zinc_data['smiles']):
  smile          = smile.replace('\n', '')
  adjusted_smiles.append(smile)

zinc_data['smiles'] = adjusted_smiles

In [7]:
zinc_data

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182
...,...,...,...,...
249450,CC1(C)CC[C@H](CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc21,3.36790,0.745901,2.900726
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,2.87430,0.799426,2.326627
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1,2.90054,0.878086,1.840642
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1,2.55624,0.852917,2.024638


**Data Details**
1. ssr contains list of lists with indexes that correspond to rings
2. Atoms contains the symbol of the atom corresponding to index
3. Bonds contains tuples of bonded indexes

**Algorithm for Junction Tree**
1. Simple version has been solved, not investigating nuances in chemical structures
2. Next step: max each node to a dictionary index value representing different molecules, rings, etc.
3. Will need to add a check for aromatic molecules so that can be added to chemical vocabulary
4. Will need to get bond types (double, triple) etc to correctly create vocabulary
5. Consider other chemical structure occurences that would ened to be considered to create a chemical dictionary of the nodes in each of our molecules
6. Last step is to map the values in adjency matrix to their true value (1 --> 103 example)

In [ ]:
# def smiles_to_matrix():
v1                   = {}
v2_ring              = {}
adjacency_matricies  = []
atom_dict            = {}
for i, smile in tqdm(enumerate(zinc_data['smiles']), total = len(zinc_data)):
  molecule       = Chem.MolFromSmiles(smile)
  atoms          = [atom.GetSymbol() for atom in molecule.GetAtoms()]

  # Store original atom indexes in dictionary for later to create molecule tokens
  atom_dict[i]   = {index: atom for index, atom in enumerate(atoms)}
  bonds          = [tuple(sorted((bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()))) for bond in molecule.GetBonds()]
  ssr            = [list(x) for x in Chem.GetSymmSSSR(molecule)]  # GetSymmSSSR finds smallest set of smallest rings
  v2             = set()
  non_ring_bonds = set(sorted(bonds))

  # Iterate Through Rings
  for ring in ssr:
    temp_atoms = copy.copy(atoms)

    for index in range(len(ring)):
      begin_atom_idx = ring[index]
      end_atom_idx   = ring[(index + 1) % len(ring)]  # To loop back to the start
      bond           = molecule.GetBondBetweenAtoms(begin_atom_idx, end_atom_idx)

      if bond is not None:
          bond_idx_tuple = tuple(sorted((begin_atom_idx, end_atom_idx)))
          v2.add(bond_idx_tuple)

    # v2
    non_ring_bonds = set(sorted(non_ring_bonds)) - set(sorted(v2))

    subset_dict        = [atoms, v2, non_ring_bonds]
    v1[i]              = non_ring_bonds

    # Stores all V2 rings after some post processing
    v2_ring[i]         = ssr

  # Step 2: Merge Bridged Rings
  # Example of Bridged Ring: Bicyclo[2.2.2]octane
  if len(ssr) > 0:
    v2_copy     = copy.deepcopy(v2_ring[i])
  # If no rings, all bonds are the bonds calculated initially
  else:
    v1[i]       = bonds

  counter = 0
  for r1 in v2_copy:
      for r2 in v2_copy:
          # Don't evaluate identical rings and fit algorithm criteria
          if r1 != r2 and len(set(r1).intersection(r2)) > 2:
              # Merge r1 and r2
              counter += 1
              new_ring       = r1 + r2
              v2_ring[i]     = [ring for ring in v2_ring[i] if ring not in [r1, r2]]
              v2_ring[i].append(new_ring)

  # Step 3: Find Intersection Atoms (V0)
  # V0 ← atoms being the intersection of three or more clusters in V1 ∪ V2
  # Property that finds tertiary carbons/ any other tertiary molecules (Mostly Carbons)
  all_clusters = list(v1[i]) + list(v2)
  atoms        = [atom for cluster in all_clusters for atom in cluster]
  atom_counts  = Counter(atoms)
  v0           = {atom for atom, count in atom_counts.items() if count >= 3}

  # The set of all nodes (V)
  V              = set().union(v1[i])
  if len(ssr) > 0:
    v2_ring[i]   = set([tuple(sub_list) for sub_list in v2_ring[i]])
    V            = V.union([ring for ring in v2_ring[i]])
  V              = V.union(v0)  # Add all atoms from v0


  # Add nodes for each cluster (ring or bond)
  junction_tree = nx.Graph()
  for cluster in V:
      junction_tree.add_node(cluster)

  # Add edges between clusters that share junction atoms
  # Finds the shared atoms for Nodes
  for cluster1 in junction_tree.nodes():
      for cluster2 in junction_tree.nodes():
          if cluster1 != cluster2:

              # Different combinations of Tertiary, single atoms, and ring bonds
              shared_atoms = None
              if isinstance(cluster1, int) and isinstance(cluster2, int):
                shared_atoms = set([cluster1]).intersection(set([cluster2]))
              elif isinstance(cluster2, int):
                  if cluster2 in cluster1:  # If the single atom is in the cluster
                      shared_atoms = set([cluster2])
              elif isinstance(cluster1, int):
                  if cluster1 in cluster2:  # If the single atom is in the cluster
                      shared_atoms = set([cluster1])
              else:
                shared_atoms = set(cluster1).intersection(set(cluster2))


              if shared_atoms is not None and len(shared_atoms) > 0:
                contains_index = any(list(shared_atoms) in tup for tup in list(v1[i]))
                # Conditions to check for consecutive v1 bonds... (1,2) should be connected to (2, 4)
                v1_bonds       = True in [list(shared_atoms)[0] in tup for tup in list(v1[i])]


              if shared_atoms and (shared_atoms.issubset(v0) or v1_bonds):
                  junction_tree.add_edge(cluster1, cluster2, weight=len(shared_atoms))

  # Now junction_tree is your junction graph representing the molecule

  # Step 5: Find Maximum Spanning Tree
  mst = nx.maximum_spanning_tree(junction_tree)

  # Get the adjacency matrix representing connections
  adj_matrix             = nx.adjacency_matrix(mst)
  adj_matrix             = adj_matrix.todense()

  adjacency_matricies.append(adj_matrix)

In [ ]:
atom_dict

**Question 01 Feb 2024**
1. WHat do the individual nodes represent vs the center nodes and why are they not connectd to the center
2. as it stands, singular atoms are not being joined to bonds in the strucutre. Look at print statement above to see what i mean future me!

In [ ]:
print("Nodes:", junction_tree.nodes(data=True))
print("Edges:", junction_tree.edges(data=True))

Nodes: [((0, 1), {}), ((9, 10), {}), (1, {}), (4, {}), (5, {}), (8, {}), ((21, 22, 23, 24, 25, 20), {}), ((19, 20), {}), (9, {}), ((12, 13), {}), ((6, 7, 8, 26, 27, 5), {}), (13, {}), (18, {}), ((8, 9), {}), (20, {}), ((9, 11), {}), ((18, 19), {}), ((11, 12), {}), ((4, 5), {}), ((1, 2, 3, 4, 28), {}), ((14, 15, 16, 17, 18, 13), {})]
Edges: [((0, 1), 1, {'weight': 1}), ((0, 1), (1, 2, 3, 4, 28), {'weight': 1}), ((9, 10), 9, {'weight': 1}), ((9, 10), (8, 9), {'weight': 1}), ((9, 10), (9, 11), {'weight': 1}), (1, (1, 2, 3, 4, 28), {'weight': 1}), (4, (4, 5), {'weight': 1}), (4, (1, 2, 3, 4, 28), {'weight': 1}), (5, (6, 7, 8, 26, 27, 5), {'weight': 1}), (5, (4, 5), {'weight': 1}), (8, (6, 7, 8, 26, 27, 5), {'weight': 1}), (8, (8, 9), {'weight': 1}), ((21, 22, 23, 24, 25, 20), (19, 20), {'weight': 1}), ((21, 22, 23, 24, 25, 20), 20, {'weight': 1}), ((19, 20), 20, {'weight': 1}), ((19, 20), (18, 19), {'weight': 1}), (9, (8, 9), {'weight': 1}), (9, (9, 11), {'weight': 1}), ((12, 13), 13, {'we

In [ ]:
# Draw the graph
plt.figure(figsize=(12, 12))  # Set the size of the plot
pos = nx.spring_layout(junction_tree)  # Positions for all nodes

# Nodes
nx.draw_networkx_nodes(junction_tree, pos, node_size=50, node_color="lightblue")

# Edges
nx.draw_networkx_edges(junction_tree, pos, width=2)

# Labels
nx.draw_networkx_labels(junction_tree, pos, font_size=6, font_family="sans-serif")

# Show the plot
plt.axis("off")  # Turn off the axis
plt.show()

**Algorithm 2**

 Tree decomposition of molecule G = (V, E)
1. V1 ← the set of bonds (u, v) ∈ E that do not belong to any rings.
2. V2 ← the set of simple rings of G.
forr1,r2 inV2 do
Merge rings r1 , r2 into one ring if they share more than two atoms (bridged rings). end for
V0 ← atoms being the intersection of three or more clusters in V1 ∪ V2.
V ← V0 ∪ V1 ∪ V2
E ← {(i, j, c) ∈ V × V × R | |i ∩ j| > 0}. Set c = ∞ if i ∈ V0 or j ∈ V0, and c = 1 otherwise. Return The maximum spanning tree over cluster graph (V , E ).